In [1]:
import re
import os
import shutil
import numpy as np

In [19]:
def get_label(name):
    """Retrieves the label from an image filename"""
    label_regex = '(\w+)_\d+.jpg'
    match = re.match(label_regex, name)
    return match.group(1)

In [33]:
your_path = 'JPEGImages/'
my_path = 'dataset/'
# splits_path_train = 'ImageSplits/train.txt'
# splits_path_test = 'ImageSplits/test.txt'

In [34]:
# files = os.listdir(your_path)
# for file in files:
#     label = get_label(file)
#     shutil.copy(your_path + file, my_path + label)

In [35]:
train_data_dir = 'dataset/train/'
test_data_dir = 'dataset/test/'

# with open(splits_path_train, 'r') as f_train:
#     for line in f_train:
#         label = get_label(line)
#         line = line.replace('\n','')
#         shutil.copy(your_path + line, train_data_dir + label + '/' + line)
#
# with open(splits_path_test, 'r') as f_test:
#     for line in f_test:
#         label = get_label(line)
#         line = line.replace('\n','')
#         shutil.copy(your_path + line, test_data_dir + label + '/' + line)

In [36]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import datasets, transforms

from torch import nn
from torch import optim

In [3]:
# data_dir = 'dataset'
#
# def load_split_train_test(datadir, valid_size = .2):
#     train_transforms = transforms.Compose([transforms.Resize([299,299]),
#                                        transforms.ToTensor(),
#                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
#
#     test_transforms = transforms.Compose([transforms.Resize([299,299]),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
#
#     train_data = datasets.ImageFolder(datadir, transform=train_transforms)
#     test_data = datasets.ImageFolder(datadir, transform=test_transforms)
#
#     num_train = len(train_data)
#     indices = list(range(num_train))
#     split = int(np.floor(valid_size * num_train))
#     np.random.shuffle(indices)
#
#     train_idx, test_idx = indices[split:], indices[:split]
#     train_sampler = SubsetRandomSampler(train_idx)
#     test_sampler = SubsetRandomSampler(test_idx)
#
#     trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=64)
#     testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=64)
#
#     return trainloader, testloader

In [ ]:
# trainloader, testloader = load_split_train_test(data_dir, .2)

In [39]:
train_transforms = transforms.Compose([transforms.Resize([299,299]),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize([299,299]),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(train_data_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_data_dir, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, num_workers=4, batch_size=64)
testloader = torch.utils.data.DataLoader(test_data, shuffle=True, num_workers=4, batch_size=64)

In [40]:
print(trainloader.dataset.classes)
print(testloader.dataset.classes)

['applauding', 'blowing_bubbles', 'brushing_teeth', 'cleaning_the_floor', 'climbing', 'cooking', 'cutting_trees', 'cutting_vegetables', 'drinking', 'feeding_a_horse', 'fishing', 'fixing_a_bike', 'fixing_a_car', 'gardening', 'holding_an_umbrella', 'jumping', 'looking_through_a_microscope', 'looking_through_a_telescope', 'phoning', 'playing_guitar', 'playing_violin', 'pouring_liquid', 'pushing_a_cart', 'reading', 'riding_a_bike', 'riding_a_horse', 'rowing_a_boat', 'running', 'shooting_an_arrow', 'smoking', 'taking_photos', 'texting_message', 'throwing_frisby', 'using_a_computer', 'walking_the_dog', 'washing_dishes', 'watching_TV', 'waving_hands', 'writing_on_a_board', 'writing_on_a_book']
['applauding', 'blowing_bubbles', 'brushing_teeth', 'cleaning_the_floor', 'climbing', 'cooking', 'cutting_trees', 'cutting_vegetables', 'drinking', 'feeding_a_horse', 'fishing', 'fixing_a_bike', 'fixing_a_car', 'gardening', 'holding_an_umbrella', 'jumping', 'looking_through_a_microscope', 'looking_throu

In [41]:
activity_model = torchvision.models.inception_v3(pretrained=True, aux_logits=False)
print('model initialized')

model initialized


In [42]:
for param in activity_model.parameters():
    param.requires_grad = False

activity_model.fc =  nn.Linear(2048, len(trainloader.dataset.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(activity_model.parameters(), lr=0.001)

In [43]:
epochs = 50
steps = 0
running_loss = 0
best_accuracy = 0
test_accuracy = 0
print_every = 10
train_losses, test_losses = [], []

path = 'saved_model/'

activity_model.cuda()

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        logps = activity_model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            activity_model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    logps = activity_model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()

                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            test_accuracy = accuracy/len(testloader)
            running_loss = 0
            activity_model.train()
    if test_accuracy > best_accuracy:
        torch.save(activity_model, path + 'activity_model.pth')
        best_accuracy = test_accuracy

Epoch 1/50.. Train loss: 3.661.. Test loss: 3.413.. Test accuracy: 0.182
Epoch 1/50.. Train loss: 3.381.. Test loss: 3.093.. Test accuracy: 0.315
Epoch 1/50.. Train loss: 3.045.. Test loss: 2.796.. Test accuracy: 0.511
Epoch 1/50.. Train loss: 2.775.. Test loss: 2.598.. Test accuracy: 0.527
Epoch 1/50.. Train loss: 2.550.. Test loss: 2.361.. Test accuracy: 0.548
Epoch 1/50.. Train loss: 2.392.. Test loss: 2.209.. Test accuracy: 0.556
Epoch 2/50.. Train loss: 2.101.. Test loss: 2.082.. Test accuracy: 0.574
Epoch 2/50.. Train loss: 2.002.. Test loss: 1.956.. Test accuracy: 0.590
Epoch 2/50.. Train loss: 1.852.. Test loss: 1.871.. Test accuracy: 0.608
Epoch 2/50.. Train loss: 1.864.. Test loss: 1.747.. Test accuracy: 0.622
Epoch 2/50.. Train loss: 1.746.. Test loss: 1.690.. Test accuracy: 0.634
Epoch 2/50.. Train loss: 1.735.. Test loss: 1.651.. Test accuracy: 0.625
Epoch 3/50.. Train loss: 1.649.. Test loss: 1.607.. Test accuracy: 0.638
Epoch 3/50.. Train loss: 1.415.. Test loss: 1.530..

KeyboardInterrupt: 